In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import sqlite3
driver = webdriver.Chrome('C:/Users/noilkwon/dev/기타/chromedriver.exe')

In [2]:
# DB 생성
conn = sqlite3.connect('auction.db')
cur = conn.cursor()

In [3]:
# 테이블 생성
#cur.execute('create table 경매(사건번호, 소재지, 가격, 매각기일, 진행상태, 조회수)')
#cur.execute('drop table 경매')
cur.execute('delete from 경매')

In [4]:
# 홈페이지 접속
url = r'http://www.courtauction.co.kr/maemul/search_total.asp'
driver.get(url)

In [5]:
# 매각기일
selector = ['table#Table1','tbody','tr:nth-of-type(2)','td','#Table4','tbody','tr','td:nth-of-type(2)','input#sdate']
selector = ' > '.join(selector)
driver.find_element_by_css_selector(selector).clear()
driver.find_element_by_css_selector(selector).send_keys('2000-01-01')

In [6]:
# 소재지
select = Select(driver.find_element_by_id('sido'))
select.select_by_visible_text('서울')

In [7]:
# 용도 복수선택
driver.find_element_by_id('Checkbox8').click()

In [8]:
# 정렬방식
select = Select(driver.find_element_by_id('sort'))
select.select_by_value('1')

In [9]:
# 버튼 클릭
driver.find_element_by_id('search').click()

In [10]:
# 함수 정의
def getText(row,col):
    selector = ['#Table22','tbody','tr:nth-of-type('+str(row)+')','td:nth-of-type('+str(col)+')']
    selector = ' > '.join(selector)
    text = driver.find_element_by_css_selector(selector).text
    return text

def getPage():
    data = []
    for r in [6,8,10,12,14,16,18,20]:
        values = []
        # 사건번호, 소재지, 가격, 매각기일, 진행상태, 조회수
        for c in [2,4,5,6,7,8]:   
            values.append(getText(r,c))
        data.append(values)
    return data

In [12]:
# 작업 시작
page = 7

for i in range(page):
    j = i//10
    driver.execute_script("GoPage("+str(j)+","+str(i+1)+",'board')")
    cur.executemany("insert into 경매 values (?,?,?,?,?,?)", getPage())
    if i%70 == 0:
        conn.commit()
        print("{}번째 페이지 적재".format(i+1))

1번째 페이지 적재


In [13]:
# 적재 데이터 출력
cur.execute("select * from 경매")
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['사건번호', '소재지', '가격', '매각기일', '진행상태', '조회수'])
df

,사건번호,소재지,가격,매각기일,진행상태,조회수
0,서울중앙지법6계\n2016-9088[1]\n근린상가,"서울 중구 을지로6가 18-21 ,-220,-221 굿모닝시티쇼핑몰 2층 F2014...","80,000,000\n13,422,000\n16.8%",2018-06-27\n(10:00),진행\n(유찰8회),6
1,서울중앙지법6계\n2016-12916[1]\n근린상가,"서울 중구 을지로6가 18-21 ,-220,-221 굿모닝시티쇼핑몰 7층 F7189...","43,000,000\n7,214,000\n16.8%",2018-06-27\n(10:00),진행\n(유찰8회),3
2,서울중앙지법6계\n2017-10412[1]\n근린상가,서울 서초구 양재동 275-2 윈드스톤호피스텔 지하1층 148호 [논현로 79]\n...,"300,000,000\n192,000,000\n64.0%",2018-06-27\n(10:00),진행\n(유찰2회),5
3,서울중앙지법6계\n2017-105389[1]\n근린상가,서울 중구 산림동 90-4 외1 [청계천로 168]\n건물1.1평 토지1.8평,"171,163,400\n136,931,000\n80.0%",2018-06-27\n(10:00),진행\n(유찰1회),7
4,서울남부지법10계\n2017-1124[1]\n근린상가,서울 구로구 구로동 3-25 신도림테크노마트 3층 289호 외1 [새말로 97]\n...,"146,000,000\n38,274,000\n26.2%",2018-06-19\n(10:00),진행\n(유찰6회),2
5,서울남부지법10계\n2017-9616[1]\n근린상가,서울 영등포구 여의도동 11-1 순복음 지하층 10호 [국회대로76가길 11]\n건...,"110,000,000\n70,400,000\n64.0%",2018-06-19\n(10:00),진행\n(유찰2회),2
6,서울남부지법10계\n2017-10180[1]\n근린상가,"서울 양천구 신정동 334 ,335 쌍용상가 2동 지하층 101호 [신목로 16]\...","25,000,000\n20,000,000\n80.0%",2018-06-19\n(10:00),진행\n(유찰1회),2
7,서울남부지법10계\n2017-100953[1]\n근린상가,서울 구로구 구로동 3-25 신도림테크노마트 10층 024호 [새말로 97]\n건물...,"720,000,000\n188,744,000\n26.2%",2018-06-19\n(10:00),진행\n(유찰6회),2
8,서울중앙지법6계\n2016-9088[1]\n근린상가,"서울 중구 을지로6가 18-21 ,-220,-221 굿모닝시티쇼핑몰 2층 F2014...","80,000,000\n13,422,000\n16.8%",2018-06-27\n(10:00),진행\n(유찰8회),6
9,서울중앙지법6계\n2016-12916[1]\n근린상가,"서울 중구 을지로6가 18-21 ,-220,-221 굿모닝시티쇼핑몰 7층 F7189...","43,000,000\n7,214,000\n16.8%",2018-06-27\n(10:00),진행\n(유찰8회),3
